# Notebook 5: Model Evaluation
As in the previous notebooks, in this  notebook you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This notebook will utilize a copy of the extracted feature dataset we have been working with. The dataset has been processed by performing outlier clipping, z-score and range scaling, and forward feature selection to select 20 features. We are now going to utilize more than one partition worth of data, so for the z-score and range scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 

---

## Step 1: Downloading the Data

This notebook will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) and will add the use of [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---

For this notebook, cleaning, transforming, and normalization of the data has been completed using both partitions to find the various minimum, maximum, standard deviation, and mean values needed to perform these operations. Recall from lecture that we should not perform these operations on each partition individually, but as a whole as there may(will) be different values for these in different partitions. 

For example, if we perform simple range scaling on each partition individually and we see a range of 0 to 100 in one partition and 0 to 10 in another. After individual scaling the values with 100 in the first would be mapped to 1 just like the values that had 10 in the second. This can cause serious performance problems in your model, so I have made sure that the normalization was treated properly for you. 

Below you will find the full partitions and `toy` sampled data from each partition, where only 20 samples from each of our 5 classes have been included in the data.  

#### Full
- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Full Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2ExtractedFeatures.csv)

#### Toy
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition2ExtractedFeatures.csv)

Now that you have the two files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

For each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

In [65]:
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [66]:
def calc_tss(y_true=None, y_predict=None):
    """
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    """
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

As in the previous notebook, we will be utilizing a binary classification of our 5 class dataset. So, below is the helper function to change our class labels from the 5 class target feature to the binary target feature. The function is implemented to take a dataframe (e.g. our `abt`) and prepares it for a binary classification by merging the `X`- and `M`-class samples into one group, and the rest (`NF`, `B`, and `C`) into another group, labeled with `1`s and `0`s, respectively.

In [67]:
def dichotomize_X_y(data: pd.DataFrame):
    """
    dichotomizes the dataset and split it into the features (X) and the labels (y).
    
    :return: two np.ndarray objects X and y.
    """
    data_dich = data.copy()
    data_dich['lab'] = data_dich['lab'].map({'NF': 0, 'B': 0, 'C': 0, 'M': 1, 'X': 1})
    y = data_dich['lab']
    X = data_dich.drop(['lab'], axis=1)
    return X.values, y.values

In [68]:
data_dir = 'data/FDS/'
data_file = "toy_normalized_partition1ExtractedFeatures.csv"
data_file2 = "toy_normalized_partition2ExtractedFeatures.csv"

In [69]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2))

---
### P1 

Just like you did with the previous notebook, you will be utilizing a few different types of feature selection to find subsets of descriptive features to use in the models we will be evaluating.  For this part you will again be utilizing the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). You will then be using 3 diferent feature evaluation functions.

-  [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif)

- [scikit-learn mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

- [chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2)

For each of these combinations of evaluation functions, you need to construct a 20 feature training and testing dataset. This will be done by:
<ol>
    <li>Use the `SelectKBest` class with each of the evaluation functions to perform feature selection using Partition 1 as your input data</li>
    <li>Construct a new train `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 1</li>
    <li>Construct a new test `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 2</li>
</ol>

After this part, you should have a total of 6 `DataFrame`s to use in later parts, a train and test pair for each feature selection method.

---

In [70]:
numFeat = 20
abt_cpy = abt.copy()
abt_cpy2 = abt2.copy()
X, y = dichotomize_X_y(abt_cpy)
X2, y2 = dichotomize_X_y(abt_cpy2)

In [71]:
#----------------------------------------------
##### f_classif features choices
select = SelectKBest(score_func=f_classif, k=numFeat)
select.fit(X, y)
feats_to_check = select.get_support()

choosen_feats1 = pd.DataFrame(abt_cpy.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats1 = choosen_feats1.join(abt_cpy.iloc[:, i+1])

# train1
train1 = choosen_feats1
del choosen_feats1

select = SelectKBest(score_func=f_classif, k=numFeat)
select.fit(X2, y2)
feats_to_check = select.get_support()

choosen_feats1 = pd.DataFrame(abt_cpy2.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats1 = choosen_feats1.join(abt_cpy2.iloc[:, i+1])

# test1
test1 = choosen_feats1

##### mutual_into_classif choices
select = SelectKBest(score_func=mutual_info_classif, k=numFeat)
select.fit(X, y)
feats_to_check = select.get_support()

choosen_feats2 = pd.DataFrame(abt_cpy.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats2 = choosen_feats2.join(abt_cpy.iloc[:, i+1])

# train2
train2 = choosen_feats2
del choosen_feats2

select = SelectKBest(score_func=mutual_info_classif, k=numFeat)
select.fit(X2, y2)
feats_to_check = select.get_support()

choosen_feats2 = pd.DataFrame(abt_cpy2.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats2 = choosen_feats2.join(abt_cpy2.iloc[:, i+1])

# test2
test2 = choosen_feats2


###### chi2 choices
select = SelectKBest(score_func=chi2, k=numFeat)
select.fit(X, y)
feats_to_check = select.get_support()

choosen_feats3 = pd.DataFrame(abt_cpy.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats3 = choosen_feats3.join(abt_cpy.iloc[:, i+1])

# train3
train3 = choosen_feats3
del choosen_feats3

select = SelectKBest(score_func=chi2, k=numFeat)
select.fit(X2, y2)
feats_to_check = select.get_support()

choosen_feats3 = pd.DataFrame(abt_cpy2.iloc[:, 0])
for i, b in enumerate(feats_to_check):
    if b:
        choosen_feats3 = choosen_feats3.join(abt_cpy2.iloc[:, i+1])

# test 3
test3 = choosen_feats3
#----------------------------------------------

---
### P2 

Now that we have our training and testing datasets for each of our feature subsets, we need to attempt to perform hyperparameter tuning on our model for each of the datasets. We want to see which combination of dataset and parameter settings seem to provide the best results. 

In order to do this, we must first dichotomize the training and testing data. Lucky for you, a method has already been provided to do this. All you need to do is apply it to teach of the `DataFrame`s you constructed in P1.  

With your binary classification dataset constructed, now it's time to start training and testing some models. We will start with the simple [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, for each of your three copies of the Partition 1 training datasets that have had their `lab` columns converted to a binary label, train 4 different instances with the following settings. **(see documentation to know what these are)** In total you will train and evaluate 12 model setting and feature selected data pairings. 

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result. **NOTE: The model does take a little while to evaluate.**

---

In [72]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [73]:
#----------------------------------------------
def knear(data, temp):
   
    X, y = dichotomize_X_y(data)

    n = KNeighborsClassifier(n_neighbors=temp[0][0], p=temp[1][0])
    n.fit(X, y)
    y_pred1 = n.predict(X)
    
    n = KNeighborsClassifier(n_neighbors=temp[0][0], p=temp[1][1])
    n.fit(X, y)
    y_pred2 = n.predict(X)

    n = KNeighborsClassifier(n_neighbors=temp[0][1], p=temp[0][0])
    n.fit(X, y)
    y_pred3 = n.predict(X)

    n = KNeighborsClassifier(n_neighbors=temp[0][1], p=temp[1][1])
    n.fit(X, y)
    y_pred4 = n.predict(X)

    return y_pred1, y_pred2, y_pred3, y_pred4 
    

# get y_preds
y_pred1_3_1, y_pred1_3_2, y_pred1_5_1, y_pred1_5_2 = knear(train1, temp)
y_pred2_3_1, y_pred2_3_2, y_pred2_5_1, y_pred2_5_2 = knear(train2, temp)
y_pred3_3_1, y_pred3_3_2, y_pred3_5_1, y_pred3_5_2 = knear(train3, temp)

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)

# Get confusion matrices
cm_list = []
cm131 = confusion_matrix(y1_test, y_pred1_3_1)
cm_list.append(cm131)
cm132 = confusion_matrix(y1_test, y_pred1_3_2)
cm_list.append(cm132)
cm151 = confusion_matrix(y1_test, y_pred1_5_1)
cm_list.append(cm151)
cm152 = confusion_matrix(y1_test, y_pred1_5_2)
cm_list.append(cm152)
cm231 = confusion_matrix(y2_test, y_pred2_3_1)
cm_list.append(cm231)
cm232 = confusion_matrix(y2_test, y_pred2_3_2)
cm_list.append(cm232)
cm251 = confusion_matrix(y2_test, y_pred2_5_1)
cm_list.append(cm251)
cm252 = confusion_matrix(y2_test, y_pred2_5_2)
cm_list.append(cm252)
cm331 = confusion_matrix(y3_test, y_pred3_3_1)
cm_list.append(cm331)
cm331 = confusion_matrix(y3_test, y_pred3_3_2)
cm_list.append(cm331)
cm351 = confusion_matrix(y3_test, y_pred3_5_1)
cm_list.append(cm351)
cm352 = confusion_matrix(y3_test, y_pred3_5_2)
cm_list.append(cm352)

# get TSSs
for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

#----------------------------------------------

TSS:  0.85
TSS:  0.8250000000000001
TSS:  0.725
TSS:  0.725
TSS:  0.7583333333333333
TSS:  0.725
TSS:  0.775
TSS:  0.7666666666666667
TSS:  0.75
TSS:  0.775
TSS:  0.75
TSS:  0.725


---
### P3 (10 points)

After evaluating the various results from P2, you will notice that the results are not all that great with greater than 1000 false negatives for nearly all of our settings tried. But, what can be done to improve our results? If you read the documentation for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), which you certainly should have, you will see that we were only using the `MinkowskiDistance` metric with different values of `p`. If you look into the [DistanceMetric](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric) documentation for the neighbors classifiers, you will see there are several others available to use.

So, for this part, train and evaluate two more instances of [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) for each of our different feature selection train test datsets, but this time using the `ChebyshevDistance` metric instead of the `MinkowskiDistance` metric.  For these models you will only be changing the number neighbors to 3 and 5, as the values of `p` are not used for the `ChebyshevDistance` metric. 

---

In [74]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [75]:
 #----------------------------------------------
def knear_cd(data, temp):
   
    X, y = dichotomize_X_y(data)

    n = KNeighborsClassifier(n_neighbors=temp[0][0], metric='chebyshev')
    n.fit(X, y)
    y_pred1 = n.predict(X)
    
    n = KNeighborsClassifier(n_neighbors=temp[0][1], metric='chebyshev')
    n.fit(X, y)
    y_pred2 = n.predict(X)

    return y_pred1, y_pred2

# get y_preds
y_pred1_3_1, y_pred1_5_1,= knear_cd(train1, temp)
y_pred2_3_1, y_pred2_5_1 = knear_cd(train2, temp)
y_pred3_3_1, y_pred3_5_1 = knear_cd(train3, temp)

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)

# Get confusion matrices
cm_list = []
cm131 = confusion_matrix(y1_test, y_pred1_3_1)
cm_list.append(cm131)

cm151 = confusion_matrix(y1_test, y_pred1_5_1)
cm_list.append(cm151)

cm231 = confusion_matrix(y2_test, y_pred2_3_1)
cm_list.append(cm231)

cm251 = confusion_matrix(y2_test, y_pred2_5_1)
cm_list.append(cm251)

cm331 = confusion_matrix(y3_test, y_pred3_3_1)
cm_list.append(cm331)

cm351 = confusion_matrix(y3_test, y_pred3_5_1)
cm_list.append(cm351)


# get TSSs
for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

#----------------------------------------------

TSS:  0.8
TSS:  0.7833333333333333
TSS:  0.7833333333333333
TSS:  0.7166666666666667
TSS:  0.8583333333333334
TSS:  0.7833333333333333


---
### P4 (10 points)

After evaluating the results from P3, you will see that the results are no better than those we found for P2. This leads to the thought that maybe the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) is just not a good fit for the problem we are applying it to. So, let's move on to another classifier for this problem. 

In this part, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, continuing to use our training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label, train 8 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [76]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [77]:
def dec_tree(data, temp):
    
    X, y = dichotomize_X_y(data)
    m = []

    clf = DecisionTreeClassifier(criterion=temp[0][0], max_depth=temp[1][0], splitter=temp[2][0])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)
    
    clf = DecisionTreeClassifier(criterion=temp[0][0], max_depth=temp[1][0], splitter=temp[2][1])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    clf = DecisionTreeClassifier(criterion=temp[0][0], max_depth=temp[1][1], splitter=temp[2][0])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    clf = DecisionTreeClassifier(criterion=temp[0][0], max_depth=temp[1][1], splitter=temp[2][1])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    ############
    clf = DecisionTreeClassifier(criterion=temp[0][1], max_depth=temp[1][0], splitter=temp[2][0])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)
    
    clf = DecisionTreeClassifier(criterion=temp[0][1], max_depth=temp[1][0], splitter=temp[2][1])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    clf = DecisionTreeClassifier(criterion=temp[0][1], max_depth=temp[1][1], splitter=temp[2][0])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    clf = DecisionTreeClassifier(criterion=temp[0][1], max_depth=temp[1][1], splitter=temp[2][1])
    clf.fit(X, y)
    y_pred = clf.predict(X)
    m.append(y_pred)

    return m

# get models 

dt_models1 = dec_tree(train1, temp)
dt_models2 = dec_tree(train2, temp)
dt_models3 = dec_tree(train3, temp)

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)


del cm_list
cm_list = []

# get confusion_matrices
for model in dt_models1:
    cm = confusion_matrix(y1_test, model)
    cm_list.append(cm)

for model in dt_models2:
    cm = confusion_matrix(y2_test, model)
    cm_list.append(cm)

for model in dt_models3:
    cm = confusion_matrix(y3_test, model)
    cm_list.append(cm)

for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

TSS:  0.9666666666666667
TSS:  0.875
TSS:  1.0
TSS:  1.0
TSS:  0.95
TSS:  0.9
TSS:  1.0
TSS:  1.0
TSS:  0.95
TSS:  0.9583333333333333
TSS:  1.0
TSS:  1.0
TSS:  0.975
TSS:  0.8916666666666666
TSS:  1.0
TSS:  1.0
TSS:  0.95
TSS:  0.8583333333333334
TSS:  1.0
TSS:  1.0
TSS:  0.9583333333333333
TSS:  0.8333333333333333
TSS:  1.0
TSS:  1.0


---
### P5 

After evaluating results from P4, you will see that the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) was able to accomplish a bit of an improvement over the best resutls we found for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).  This is indeed great, but can we do better than this if we use yet another classifier? Let's move on to yet another and find out.

For this part you will be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. We won't be changing any of the default settings, just train 1 model for each of our feature selected data subsets. You will again be using your training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label. You will then test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [78]:
def gnb(data):

    X, y = dichotomize_X_y(data)
    
    clf = GaussianNB()
    clf.fit(X, y)
    y_pred = clf.predict(X)
    return y_pred

# get y_preds models
gnb_models = []
gnb_models.append(gnb(train1))
gnb_models.append(gnb(train2))
gnb_models.append(gnb(train3))

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)

# get confusion matrices
del cm_list 
cm_list = []
cm1 = confusion_matrix(y1_test, gnb_models[0])
cm_list.append(cm1)
cm2 = confusion_matrix(y1_test, gnb_models[1])
cm_list.append(cm2)
cm3 = confusion_matrix(y1_test, gnb_models[2])
cm_list.append(cm3)

for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

TSS:  0.7666666666666667
TSS:  0.5333333333333333
TSS:  0.6


---
### P6

If you recall from a lecture some time back, it was shown that another way of improving the results of classification is to perform some form of sampling to balance the number of samples there are for the various classes. The reason why this works for specific classifiers, and methods for doing the sampling, are numerious and we don't have enough time to cover all of them in this course.  However, it is still beneficial to know this works and that it is something that you should be considering when you are training models.  

So, for this part, we will implement a very naive method for sampling so we can use the results for training our models again.  Below you will find a function stub, complete the function and have it return a copy of the input dataframe where each class (except for the smallest one) have been undersampled to match the size of the smallest class in the dataset. In this function you should assume the `lab` column is the class label and not the dicotomized binary classification converted label.

To do this you may want to use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function of the DataFrame to get groups of rows from your DataFrame.  You may also wish to use the [sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) function to select a number of rows from a group. You can also use the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method to process each group from your grouped rows. These are just hints, you can solve the problem how you see fit.

Once this function is complete, apply it to each of your training datasets that have been constructed with different feature selection methods from partition 1 (the ones with all the NF, C, .., X labels). You will not be applying this to your testing sets. After you have your sampled feature selected datasets, you will then apply your function that converts the multi-class problem to a binary problem to each of the resultant selected subsets so we can use these new undersampled data for the next several parts.

---

In [96]:
def perform_under_sample(data:DataFrame)->DataFrame:
    #----------------------------------------------
    ## Group and find indicies of each type of lab
    df = data.copy()
    # df.groupby(by=['lab'])
    B = df[df.lab == 'B'].index.values
    C = df[df.lab == 'C'].index.values
    M = df[df.lab == 'M'].index.values
    NF = df[df.lab == 'NF'].index.values
    X = df[df.lab == 'X'].index.values

    ## gather all indices of nonflaring and flaring indices
    nonflaring = np.concatenate((NF, B, C))
    flaring = np.concatenate((X, M))
    random_indices = np.random.choice(nonflaring, len(flaring), replace=True)

    # undersample 
    under_sample_indices = np.concatenate((nonflaring,random_indices))
    under_sample = df.loc[under_sample_indices]

    return under_sample
    #----------------------------------------------

In [97]:
under_train1 = perform_under_sample(train1)
under_train2 = perform_under_sample(train1)
under_train3 = perform_under_sample(train1)
print(under_train1)


   lab  ABSNJZH_var  SAVNCPP_median  SAVNCPP_mean  SAVNCPP_var  \
60  NF     0.626705        0.882830      0.884792     0.911350   
61  NF     0.512074        0.896668      0.895379     0.898215   
62  NF     0.525705        0.871603      0.872441     0.897351   
63  NF     0.708280        0.899064      0.899471     0.940299   
64  NF     0.248296        0.805085      0.808213     0.836438   
..  ..          ...             ...           ...          ...   
25   C     0.565503        0.878911      0.884029     0.913592   
20   C     0.593854        0.890877      0.890867     0.921325   
6    B     0.515842        0.878527      0.877806     0.898744   
22   C     0.567849        0.891361      0.892639     0.900660   
8    B     0.672086        0.945807      0.945975     0.930426   

    SAVNCPP_linear_weighted_average  SAVNCPP_quadratic_weighted_average  \
60                         0.882273                            0.880605   
61                         0.894074                      

---
### P7

For this part repeat what you did for P2, but with your balanced binary classification datasets constructed in P6, uese the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. 

So, train 4 different instances with the following settings for each of your feature selected subsets, for a total of 12 different evaluations. **(see documentation to know what these are)**

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced). You shall then calculate and print the TSS score for each result. **NOTE: The model now takes less time to evaluate!**

---

In [98]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [99]:
#----------------------------------------------
# get y_preds
y_pred1_3_1, y_pred1_3_2, y_pred1_5_1, y_pred1_5_2 = knear(under_train1, temp)
y_pred2_3_1, y_pred2_3_2, y_pred2_5_1, y_pred2_5_2 = knear(under_train2, temp)
y_pred3_3_1, y_pred3_3_2, y_pred3_5_1, y_pred3_5_2 = knear(under_train3, temp)

# get y_test and make same size
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)


# Get confusion matrices
cm_list = []
cm131 = confusion_matrix(y1_test, y_pred1_3_1)
cm_list.append(cm131)
cm132 = confusion_matrix(y1_test, y_pred1_3_2)
cm_list.append(cm132)
cm151 = confusion_matrix(y1_test, y_pred1_5_1)
cm_list.append(cm151)
cm152 = confusion_matrix(y1_test, y_pred1_5_2)
cm_list.append(cm152)
cm231 = confusion_matrix(y2_test, y_pred2_3_1)
cm_list.append(cm231)
cm232 = confusion_matrix(y2_test, y_pred2_3_2)
cm_list.append(cm232)
cm251 = confusion_matrix(y2_test, y_pred2_5_1)
cm_list.append(cm251)
cm252 = confusion_matrix(y2_test, y_pred2_5_2)
cm_list.append(cm252)
cm331 = confusion_matrix(y3_test, y_pred3_3_1)
cm_list.append(cm331)
cm331 = confusion_matrix(y3_test, y_pred3_3_2)
cm_list.append(cm331)
cm351 = confusion_matrix(y3_test, y_pred3_5_1)
cm_list.append(cm351)
cm352 = confusion_matrix(y3_test, y_pred3_5_2)
cm_list.append(cm352)

# get TSSs
for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

#----------------------------------------------

TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0


---
### P8

After evaluating the various results from P7, you will notice that some of the results are improved over the same experiments we conducted in P2. Additionally, you should also notice a improvement in the speed at which the results were obtained. The question now is will we continue to see these improvements for all of our experiments? So, let's move on and see.

For this question, you will repeat the experiments from P3, but using the balanced binary classification datasets constructed in P6. You will still be using the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) like you did in P7, but you will again be changing from using the `MinkowskiDistance` metric with different values of `p` to using the `ChebyshevDistance` metric. You will construct two models for each of your feature selected datasets by changing the number neighbors to 3 and 5.

Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced), then calculate and print the TSS score for each result. 

---

In [88]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [89]:
#----------------------------------------------
def knear_cd(data, temp):
   
    X, y = dichotomize_X_y(data)

    n = KNeighborsClassifier(n_neighbors=temp[0][0], metric='chebyshev')
    n.fit(X, y)
    y_pred1 = n.predict(X)
    
    n = KNeighborsClassifier(n_neighbors=temp[0][1], metric='chebyshev')
    n.fit(X, y)
    y_pred2 = n.predict(X)

    return y_pred1, y_pred2

# get y_preds
y_pred1_3_1, y_pred1_5_1,= knear_cd(under_train1, temp)
y_pred2_3_1, y_pred2_5_1 = knear_cd(under_train2, temp)
y_pred3_3_1, y_pred3_5_1 = knear_cd(under_train3, temp)

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)

# Get confusion matrices
cm_list = []
cm131 = confusion_matrix(y1_test, y_pred1_3_1)
cm_list.append(cm131)

cm151 = confusion_matrix(y1_test, y_pred1_5_1)
cm_list.append(cm151)

cm231 = confusion_matrix(y2_test, y_pred2_3_1)
cm_list.append(cm231)

cm251 = confusion_matrix(y2_test, y_pred2_5_1)
cm_list.append(cm251)

cm331 = confusion_matrix(y3_test, y_pred3_3_1)
cm_list.append(cm331)

cm351 = confusion_matrix(y3_test, y_pred3_5_1)
cm_list.append(cm351)


# get TSSs
for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)

#----------------------------------------------

TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0


---
### P9

After evaluating the results of P8 things are looking a little less encouraging, since none of those results look to be better than the results of P7. However, the results from P3 weren't really any better than P2 in the first place, so not all is lost.  Let's continue on and see how things turn out with models like we used in P4 since those were actaully an improvement over P2 originally.

So, in this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), like you did in P4, and try several different settings to see how/if using different settings will improve our score. The difference will again be that you are now using the balanced binary classification datasets constructed in P6 to train 8 different instances for each of your feature selected datasets using the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score for each result. 

---

In [92]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [93]:
#----------------------------------------------
# get models 

dt_models1 = dec_tree(under_train1, temp)
dt_models2 = dec_tree(under_train2, temp)
dt_models3 = dec_tree(under_train3, temp)

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)


del cm_list
cm_list = []

# get confusion_matrices
for model in dt_models1:
    cm = confusion_matrix(y1_test, model)
    cm_list.append(cm)

for model in dt_models2:
    cm = confusion_matrix(y2_test, model)
    cm_list.append(cm)

for model in dt_models3:
    cm = confusion_matrix(y3_test, model)
    cm_list.append(cm)

for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)
#----------------------------------------------

TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0
TSS:  0.0


---
### P10

Unlike with [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), it seems that the sampling didn't really help much for the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).  Where before we saw a 3X improvement with the Decision Tree over the KNN classifier, we now see similar results for both classifiers.  Let's see how switching to the sampled data affectes our best performing classifier when we were using the full dataset.

For this question you will again be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier as you did in P5 but using your balanced binary classification dataset constructed in P6 to train just 1 model for each feature selected dataset. Once you have done that, test the model using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score. 

---

In [94]:
#----------------------------------------------
# get y_preds models
gnb_models = []
gnb_models.append(gnb(under_train1))
gnb_models.append(gnb(under_train2))
gnb_models.append(gnb(under_train3))

# get y_tests
X1_test, y1_test = dichotomize_X_y(test1)
X2_test, y2_test = dichotomize_X_y(test2)
X3_test, y3_test = dichotomize_X_y(test3)

# get confusion matrices
del cm_list 
cm_list = []
cm1 = confusion_matrix(y1_test, gnb_models[0])
cm_list.append(cm1)
cm2 = confusion_matrix(y1_test, gnb_models[1])
cm_list.append(cm2)
cm3 = confusion_matrix(y1_test, gnb_models[2])
cm_list.append(cm3)

for cm in cm_list:
    tn, fp, fn, tp = cm.ravel()
    tpr = tp/(tp+fn)
    tnr = tn/(tn+fp)
    fpr = fp/(fp+tn)
    tss = tpr - fpr
    print('TSS: ', tss)
#----------------------------------------------

TSS:  0.0
TSS:  0.0
TSS:  0.0


Unfortunately, we don't see much improvement for our [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. We Might have to try something different or start over and find more to model on.